<a href="https://colab.research.google.com/github/aayush1693/Text2Video-CogVideoX/blob/main/CogVideoX_2b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Colab Notebook: Text-to-Video Generation using CogVideo**


---
This notebook guides you through installing and using the CogVideo model for generating videos from text prompts. We will install the necessary libraries, set up the model, and then use it to generate a video based on a text description.


# Step 1: Install Required Libraries
In this step, we will clone the CogVideo repository and install the required Python packages.

In [2]:
# Clone the CogVideo repository from GitHub
!git clone https://github.com/THUDM/CogVideo

# Install the required dependencies from the requirements.txt file
!pip install -r CogVideo/requirements.txt

# Install Hugging Face Hub for easy access to pre-trained models
!pip install huggingface_hub

fatal: destination path 'CogVideo' already exists and is not an empty directory.


# Step 2: Set Up the Environment
We'll make sure the CogVideo module is accessible in our Python path by adding it manually. Additionally, we'll check if a GPU is available (which is highly recommended for this task).

In [3]:
import sys
import os
import torch

# Add CogVideo to Python path
sys.path.append('/content/CogVideo')  # Adjust this path if required

# Set an environment variable to avoid memory fragmentation issues
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Check if a GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU")


GPU is available


# Step 3: Import Required Libraries
Next, we'll import the necessary modules for video generation and memory management.

Note: Clearing the CUDA cache is important to avoid memory overflow during large operations like video generation.

In [4]:
import IPython.display as display
import numpy as np
from torch.quantization import quantize_dynamic

# Import necessary classes from the inference module
from inference.cli_demo import generate_video  # Adjusted import based on the directory structure
from diffusers import CogVideoXPipeline, CogVideoXVideoToVideoPipeline

# Clear CUDA cache to free up memory
torch.cuda.empty_cache()

# Step 4: Model Quantization for Efficiency
Here, we define a function to quantize the model using dynamic quantization to reduce memory usage and improve efficiency.

Explanation: Quantization helps reduce the memory footprint of the model, which is particularly useful when working in environments with limited resources, such as Google Colab.

In [5]:
# Define the quantization function to optimize the model for lower memory usage
def quantize_model(model):
    model = quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)
    return model

# Step 5: Define the Text-to-Video Generation Function
In this section, we define the main function that will generate the video based on a given text prompt.

In [6]:
from torch.cuda.amp import autocast

def generate_video_wrapper(prompt, model_id="THUDM/CogVideoX-2b", dtype=torch.float16, num_frames=30):
    """
    Generates a video based on the text prompt using CogVideo with optimizations for Colab T4.

    Args:
    - prompt (str): The text description for video generation.
    - model_id (str): The Hugging Face model ID. Defaults to "THUDM/CogVideoX-2b".
    - dtype (torch.dtype): The data type for the model. Defaults to torch.float16.
    - num_frames (int): Number of frames to generate.

    Returns:
    - numpy array: Video in NumPy format for display.
    """
    try:
        # Load the pipeline directly from the Hugging Face Hub
        pipe = CogVideoXPipeline.from_pretrained(model_id, torch_dtype=dtype).to(device)

        # Quantize the model to save memory
        pipe = quantize_model(pipe)

        # List to hold generated frames
        video_frames = []

        # Generate frames iteratively to avoid memory overflow
        for frame_idx in range(num_frames):
            with autocast():
                # Generate the current frame
                video_tensor = pipe(prompt).frames.to(device)

            # Convert tensor to NumPy array
            video_np = video_tensor.cpu().numpy()
            video_frames.append(np.clip(video_np * 255, 0, 255).astype(np.uint8))

            # Clear cache after each frame to prevent memory fragmentation
            torch.cuda.empty_cache()

        # Combine frames into a single video
        video_np = np.stack(video_frames)
        return video_np

    except Exception as e:
        print(f"Error during video generation: {e}")
        return None

Explanation:



*   This function takes a text prompt, uses the CogVideo model to generate frames iteratively (to avoid memory issues), and finally combines them into a single video.
*   We use torch.cuda.amp.autocast() to speed up computation and reduce memory usage.




# Step 6: Test the Model with a Sample Prompt
Now, let's generate a video based on a sample text description

In [7]:
# Example text prompt for video generation
prompt = "A beautiful waterfall in a forest"

# Generate a video with 20 frames (can be adjusted based on available resources)
video = generate_video_wrapper(prompt, num_frames=5)

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Error during video generation: CUDA out of memory. Tried to allocate 32.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 19.06 MiB is free. Process 18576 has 14.73 GiB memory in use. Of the allocated memory 14.61 GiB is allocated by PyTorch, and 18.56 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


Explanation: We use the generate_video_wrapper() function to create a video based on the text prompt "A beautiful waterfall in a forest." The number of frames is reduced to 5 to avoid potential memory issues on Colab. But unfortunately we get CUDA out of memory error despite all efforts. May be this code should work fine on Colab Pro or a higher end NVIDIA GPU which unfortunately I dont have access to. I promise I will update this notebook once I have such access

# Step 7: Display the Generated Video
Finally, let's display the generated video directly in the notebook.

In [ ]:
# If video was generated successfully, display it in the notebook
if video is not None:
    display.display(display.Video(video, embed=True))
else:
    print("Failed to generate video.")

Explanation: We use IPython.display to embed and display the generated video directly within the notebook. If the video generation fails, an error message is printed.

P.S: Sorry we didnt see the Video

# ***Conclusion***
In this notebook, we successfully set up and used the CogVideo model to generate a video based on a text prompt. We've also optimized the model using quantization to reduce memory usage and ensure smooth execution in a limited resource environment like Google Colab, but unfortunately we are limited by the hardware capabilities in Colab Free Tier.